<a href="https://cognitiveclass.ai/">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Logo/SNLogo.png" width="200" align="center">
</a>

<h2>Deploying Your Computer Vision Model on IBM Cloud</h2>
<p>For the capstone project of this course, you will be training a custom classifier using Watson Visual Recognition to identify furniture, in particular <b>Tables</b>, <b>Beds</b> and <b>Chairs</b>. After training your model you are then going to connect your trained custom classifier to a <b>Flask</b> app. <b>Flask</b> is a python web framework used for making web apps. You are then going to deploy your <b>Flask</b> app to IBM Cloud. Once your app is deployed on IBM Cloud you can access your <b>Flask</b> app via a webpage anywhere using a custom link.</p>

<p>This link will be used by your peers to assess your project. In your web app, your peers will be able to upload an image, which will then be classified using your custom classifier you connected in the web app. Your project will be graded by how accurately your app can classify <b>Tables</b>, <b>Beds</b> and <b>Chairs</b>.<p>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size="3"><strong>Click on the links to go to the following sections:</strong></font>
<br>
<h2>Table of Contents</h2>
<ol>
    <li><a href="#ref1">Part 1 - Training Your Custom Model</a></li>
    <li><a href="#ref2">Part 2 - Downloading and Configuring the Flask Web App</a></li>
    <li><a href="#ref4">Part 3 - Deploying Your Web App to IBM Cloud</a></li>
</ol>    
</div>

<div style="background-color: #fcf2f2">
<h2>Note!</h2> 
<p>Since the lite plan only allows for  a maximum of 2 custom classifiers, you should not have more than 1 custom classifier running in your Watson Visual Recognition Instance. You might want to delete one of your custom classifier from your Watson Visual Recognition Instance.</p>
<p>Log into your IBM Cloud Account with the following link - <a href="https://cocl.us/CV0101EN_IBM_Cloud_Login">https://cloud.ibm.com</a></p>
<ol>
    <li>Click on Services</li>
    <li>Under Services, select your Watson Visual Recognition Instance</li>
    <li> Click on Show Credentials, and note the API Key. We will be using this API Key to connect your custom classifier to your web app </li>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/API_Key.png" width="680">
    <li><b>(Optional)</b> Delete one of your custom classifier if you have 2 of them in your Watson Visual Recognition Instance</li>
</ol>
<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/Delete-Instance.png" width="680">
</div>

<a id="ref1"></a>
<h2>Part 1 - Training Your Custom Model</h2>
<p>For the first part of your capstone project, you have to obtain a dataset of <b>Tables</b>, <b>Beds</b> and <b>Chairs</b>. Next, you need to create a custom classifier based upon this dataset and train your custom model with this dataset. Creating and training a custom classifier has been covered in previous modules of this course. Make sure your custom visual recognition model classifies <b>Tables</b>, <b>Beds</b> and <b>Chairs</b> correctly, because <b>25%</b> of your grade for this project will be determined by if your trained custom classifier can identify a piece of furniture correctly with a high level of confidence. Also note the classifier id for this custom classifier as you will use this to connect your web app to your custom classifier in the next part.</p>

<a id="ref1"></a>
<h2>IBM Watson Package</h2>
Launch IBM Watson and OpenCV


In [37]:
! pip install --upgrade ibm-watson opencv-python

Requirement already up-to-date: ibm-watson in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (4.1.0)
Requirement already up-to-date: opencv-python in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (4.1.2.30)


<a id="ref2"></a>
<h2>Plotting images in Jupyter Notebooks</h2>
Function for plotting image

In [38]:
import cv2
import urllib.request
from matplotlib import pyplot as plt
from pylab import rcParams

def plt_image(image_url, size = (10,8)):

    # Downloads an image from a URL, and displays it in the notebook
    urllib.request.urlretrieve(image_url, "image.jpg") # downloads file as "image.jpg"
    image = cv2.imread("image.jpg")
    
    # If image is in color, then correct color coding from BGR to RGB
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    rcParams['figure.figsize'] = size[0], size[1] #set image display size

    plt.axis("off")
    plt.imshow(image, cmap="Greys_r")
    plt.show()

Exampe image plot

# IBM Watson API key

In [39]:
my_apikey = 'qzFYiynZB-GBBhMsnIdOEVOHXHqXdchFgNYbLVFX2maV'

<h4>Initialize Watson Visual Recognition</h4>

In [40]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2019-12-10', 
                             authenticator=authenticator)

# Training Classifier

Image recognition model will be trained with 'table', 'chair' and 'bed' image dataset into following files:
<ul>
    <li>table.zip</li>
    <li>chair.zip</li>
    <li>bed.zip</li>
</ul>
All the images were downloaded from various websites in Google. The files were categorized, zipped and uploaded in the current directory of 'Skills Network Labs'

<p>Lets train the Visual Recognition model to recognize the three furniture types using the <b>create_classifier</b> method from the Watson Image Reconition API.</p>

In [70]:
# deleting any excess classifier running in excess of 2
import json

classifiers = visrec.list_classifiers(verbose=True).get_result()['classifiers']
print(json.dumps(classifiers, indent=2))


[
  {
    "classifier_id": "furnitureclassifier_1136413941",
    "name": "furnitureclassifier",
    "status": "ready",
    "owner": "f135aaec-7451-4d18-ab62-886846cd563e",
    "created": "2019-12-10T10:01:12.549Z",
    "updated": "2019-12-10T10:01:12.549Z",
    "classes": [
      {
        "class": "bed"
      },
      {
        "class": "chair"
      },
      {
        "class": "table"
      }
    ],
    "core_ml_enabled": true
  }
]


In [69]:
# mycid = 'furnitureclassifier_640381817' #the classifier id you want to delete
# visrec.delete_classifier(classifier_id = mycid)

In [54]:
import json
with open('table.zip', 'rb') as table, \
     open('chair.zip', 'rb') as chair, \
     open('bed.zip', 'rb') as bed:
        response = visrec.create_classifier(name="furnitureclassifier",
                                        positive_examples={'table': table, \
                                                           'chair': chair, \
                                                           'bed': bed})
print(json.dumps(response.get_result(), indent=2))

{
  "classifier_id": "furnitureclassifier_1136413941",
  "name": "furnitureclassifier",
  "status": "training",
  "owner": "f135aaec-7451-4d18-ab62-886846cd563e",
  "created": "2019-12-10T10:01:12.549Z",
  "updated": "2019-12-10T10:01:12.549Z",
  "classes": [
    {
      "class": "bed"
    },
    {
      "class": "chair"
    },
    {
      "class": "table"
    }
  ],
  "core_ml_enabled": true
}


In [80]:
#Grabbing the classifier id
classifier_id = response.get_result()["classifier_id"]
classifier_id

'furnitureclassifier_1136413941'

In [81]:
# Checking if the model is still training
visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']

'ready'

In [82]:
# List all the classifiers
visrec.list_classifiers(verbose=True).get_result()

{'classifiers': [{'classifier_id': 'furnitureclassifier_1136413941',
   'name': 'furnitureclassifier',
   'status': 'ready',
   'owner': 'f135aaec-7451-4d18-ab62-886846cd563e',
   'created': '2019-12-10T10:01:12.549Z',
   'updated': '2019-12-10T10:01:12.549Z',
   'classes': [{'class': 'bed'}, {'class': 'chair'}, {'class': 'table'}],
   'core_ml_enabled': True}]}

<a id="ref4"></a>
<h2>Testing Classifier</h2>

In [85]:
from pandas.io.json import json_normalize

def getdf_visrec(url, classifier_ids, apikey = my_apikey):
    
    json_result = visrec.classify(url=url,
                              threshold='0.6',
                              classifier_ids=classifier_ids).get_result()
    
    json_classes = json_result['images'][0]['classifiers'][0]['classes']
    
    df = json_normalize(json_classes).sort_values('score', ascending=False).reset_index(drop=True)
    
    return df

In [86]:
getdf_visrec(url = 'https://labs.cognitiveclass.ai/tools/jupyterlab/lab/tree/labs/CV0101EN/table_test.jpg',
            classifier_ids=classifier_id)

ERROR:root:Unknown error
Traceback (most recent call last):
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 157, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Unknown error, Code: 400 , X-global-transaction-id: b6357be5d0e5f31eec92f6363fdb563c


ApiException: Error: Unknown error, Code: 400 , X-global-transaction-id: b6357be5d0e5f31eec92f6363fdb563c

<a id="ref2"></a>
<h2>Part 2 - Downloading and Configuring the Flask Web Appp</h2>
<p>Once you are done training your custom classifier, you will have to deploy it on the web and provide a graphical user interface for people to interact with it. <b>Flask</b> is a lightweight web framework that allows us to do that. You will connect the web app with your custom classifier and deploy the app to IBM Cloud. By deploying the app to IBM Cloud, you could share your web app with your peers by simply sharing a link with them. Make sure complete this step successfully, since another <b> 10% </b> of your grade for the capstone project depends on completion of this step</p>

<p>The code below downloads and unzips the Flask web app</p>

<h3>Downloading your Flask Appp</h3>

In [87]:
!pip install wget

  Stored in directory: /home/jupyterlab/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [88]:
import wget, zipfile, os

filename = wget.download('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/CV0101/Project/CV0101EN-capstone-project-new.zip')
with zipfile.ZipFile(filename,"r") as zip_ref:
    zip_ref.extractall()

<h3>Configuring Your Flask App</h3>
<p>In order to deploy your custom classifier, you need to give your Flask app permission to access it.</p>

<p>At this point, you should see a folder of your web app (CV0101EN-capstone-project) listed in the files directory in the left-sidebar of the JupyterLab environment. If this sidemenu is hidden, you can go to <code>View>View Left-Sidebar</code>. Select the <code>CV0101EN-capstone-project</code>, and click on the <code>app</code> subfolder. You will see a file called <b>config.py</b>. Fill in <code>API_KEY</code> with your Watson Visual Recognition API_KEY, and <code>CLASSIFIER_ID</code> you noted down in part 1 of this notebook</p>

#### Next, you will see a file <code>manifest.yml</code> Open the file, and make sure to change your app name to a name you'd like to give your app. Do NOT change anything else

<a id="ref4"></a>
<h2>Part 3 - Deploying Your Web App to IBM Cloud</h2>
<p> For the last part of your capstone project, you will deploy your web app to IBM Cloud in order for your peers to access it. Your peers will upload pictures of furniture in your app and will test the accuracy of your custom trained model. You will share a custom link of your web app with your peers in order for them to test the accuracy of your model. Deploying your project successfully is worth another<b> 15% </b> of your grade. The peer evaluation is worth <b> 50% </b> of your grade
    
<p>In order to deploy your project to the cloud, in the code cell below replace <code>email</code> and <code>password</code> with your email and password that you use for logging in to IBM Cloud
</p>

After replacing the values of `YOUR_IBMCLOUD_EMAIL` with your IBM Cloud email and `YOUR_IBMCLOUD_PASSWORD` with your password in the cell below, press "ctrl + enter" to login to your IBM Cloud account from the notebook

In [90]:
%%bash
ibmcloud config --check-version=false
ibmcloud login --no-region
saimoom_026@yahoo.com
1246o6o6O26

API endpoint: https://cloud.ibm.com

Email> saimoom_026@yahoo.com

Password> 
Authenticating...
OK

Targeted account Memorial University of Newfoundland (ac3c862b72bd49ebb4d0f14243491070) <-> 2002648

                      
API endpoint:      https://cloud.ibm.com   
Region:               
User:              saimoom_026@yahoo.com   
Account:           Memorial University of Newfoundland (ac3c862b72bd49ebb4d0f14243491070) <-> 2002648   
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'   
CF API endpoint:      
Org:                  
Space:                

Tip: If you are managing Cloud Foundry applications and services
- Use 'ibmcloud target --cf' to target Cloud Foundry org/space interactively, or use 'ibmcloud target --cf-api ENDPOINT -o ORG -s SPACE' to target the org/space.
- Use 'ibmcloud cf' if you want to run the Cloud Foundry CLI with current IBM Cloud CLI context.



<br>
<p>After the above cell finishes execution, run the code cell below. You will see your IBM cloud email listed under names. <b>Note the value of region corresponding to your email</b>. Your output should look something like this</p> <br>

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/Cloud_Foundry_Region.png" width="680">

In [91]:
%%bash
ibmcloud account orgs

Getting orgs in all regions as saimoom_026@yahoo.com...
Retrieving current account...
OK

Name                    Region     Account owner           Account ID                         Status   
saimoom_026@yahoo.com   us-south   saimoom_026@yahoo.com   ac3c862b72bd49ebb4d0f14243491070   active   



<br>
<p> Next, replace both the values of <b>REGION</b> (i.e. in <b>https://api.REGION.cf.cloud.ibm.com</b> and <b>-r REGION</b>) in the code cell below with the value you noted above. For eg, if your region was <b>eu-gb</b> you would replace both values of <b>REGION</b> with <b>eu-gb</b> in the code cell below. Next, replace <b>ORG</b> with your <b>IBM Cloud email</b>. After doing this, kindly press "ctrl+enter" to execute your code cell. <b>Kindly run the below code cell ONLY ONCE</b> <br>

In [92]:
%%bash
ibmcloud target --cf-api https://api.us-south.cf.cloud.ibm.com -r us-south -o saimoom_026@yahoo.com
ibmcloud account space-create computer-vision-app

Switched to region us-south

Targeted Cloud Foundry (https://api.us-south.cf.cloud.ibm.com)

Targeted org saimoom_026@yahoo.com


                      
API endpoint:      https://cloud.ibm.com   
Region:            us-south   
User:              saimoom_026@yahoo.com   
Account:           Memorial University of Newfoundland (ac3c862b72bd49ebb4d0f14243491070) <-> 2002648   
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'   
CF API endpoint:   https://api.us-south.cf.cloud.ibm.com (API version: 2.142.0)   
Org:               saimoom_026@yahoo.com   
Space:                

Tip: If you are managing Cloud Foundry applications and services
- Use 'ibmcloud target --cf' to target Cloud Foundry org/space interactively, or use 'ibmcloud target --cf-api ENDPOINT -o ORG -s SPACE' to target the org/space.
- Use 'ibmcloud cf' if you want to run the Cloud Foundry CLI with current IBM Cloud CLI context.

Creating space computer-vision-app in org saimoom_026@yah

<p>After the above code cell finishes execution, run the code cell below </p> 

In [93]:
%%bash
ibmcloud target -s computer-vision-app

Targeted space computer-vision-app


                      
API endpoint:      https://cloud.ibm.com   
Region:            us-south   
User:              saimoom_026@yahoo.com   
Account:           Memorial University of Newfoundland (ac3c862b72bd49ebb4d0f14243491070) <-> 2002648   
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'   
CF API endpoint:   https://api.us-south.cf.cloud.ibm.com (API version: 2.142.0)   
Org:               saimoom_026@yahoo.com   
Space:             computer-vision-app   


<p> Finally, run the code cell below to push your app to IBM Cloud. <b>Kindly run the code cell below. The cell might take a few minutes to run</b> </p>

In [94]:
%%bash
cd /resources/labs/CV0101EN/CV0101EN-capstone-project/app
ibmcloud app push

Invoking 'cf push'...

Pushing from manifest to org saimoom_026@yahoo.com / space computer-vision-app as saimoom_026@yahoo.com...
Using manifest file /resources/labs/CV0101EN/CV0101EN-capstone-project/app/manifest.yml
Getting app info...
Creating app with these attributes...
+ name:       visual recognition of furnitures
  path:       /resources/labs/CV0101EN/CV0101EN-capstone-project/app
+ memory:     128M
  routes:
+   visual-recognition-of-furnitures.mybluemix.net

Creating app visual recognition of furnitures...
Mapping routes...
Comparing local files to remote cache...
Packaging files to upload...
Uploading files...
 14.77 KiB / 14.77 KiB  100.00% 1s

Waiting for API to complete processing files...

Staging app and tracing logs...
   Downloaded liberty-for-java
   Downloaded ruby_buildpack
   Downloaded swift_buildpack_v2_0_18-20190303-1915
   Downloaded liberty-for-java_v3_38-20191031-1433
   Downloaded noop-buildpack
   Downloaded liberty_v3_14-20171013-1023
   Downloaded xpages

<br>After the above code cell finishes execution, you will see the following output towards the bottom. Kindly **_note_** the route as this will be the route you will be sharing with your peers for peer evaluation


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/app-routes.png" width="680">

<br>
Log into your IBM Cloud Account with the following link - <a href="https://cocl.us/CV0101EN_IBM_Cloud_Login">https://cloud.ibm.com</a>

<br>
<p> After logging in, Click on Cloud Foundry Apps and you will see your web app deployed to IBM Cloud. Click on it, and click on <b>Visit App URL</b> </p>

<br>
<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/cv_project_1.png" width="680">

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/cv_project_2.png" width="680">

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/cv_project_3.png" width="680">



<p>The <b>Visit App URL</b> link in cloud foundry contains the url to your Visual Recognition app</p>

<p>The url takes you to a webpage where you can upload a picture of a <b>Table</b>, <b>Bed</b> or <b>Chair</b> and it should classify the picture in their respective category with a confidence score.</p>

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/cv_project_4.png" width="680">



<br>
<p> <b> You can share this url (should be the same as the route that you noted earlier) with your peers for them to evaluate your custom classifier </b> </p>
<h1>Note!</h1>
<p>You app will be only able to classify image file in .gif, .jpg, .png and .tif format, and will not return any result if the image is in any other format.</p>
    
<h4>Kindly upload a single image at a time to the web app<h4>

<h1>Note!</h1>

If for some reason you want to redeploy your app with certain changes, **_ONLY_** execute the following cell again. **There is no need to rerun the entire notebook**

<br>
<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/redeploy-app.png" width="680">

<br>
Sample scenarios when you might want to redeploy your app:
<br>
<ol>
<li> You entered the api key / classifier id of your classifier incorrectly in <code>config.py</code> </li>
<li> You want to redeploy your app with a new name. If you redeploy your app with a new name, kindly make sure you are sharing the correct url for the web app with your peers</li>
</ol>


<h2>Thank you for completing this notebook!</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>Get IBM Watson Studio free of charge!</h2>
<p><a href="https://cocl.us/NotebooksPython101bottom"><img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Logo/BottomAd.png" width="750" align="center"></a></p>
</div>

<h3>About the Authors:</h3>
<p>This lab was written by <a href="https://www.linkedin.com/in/sacchitchadha/" target="_blank" >Sacchit Chadha</a>, and <a href="https://www.linkedin.com/in/yi-leng-yao-84451275/" target="_blank" >Yi Yao</a>.</p>
<p><a href="https://www.linkedin.com/in/sacchitchadha/" target="_blank">Sacchit Chadha</a> is a Software Engineer at IBM, and is currently pursuing a Bachelors Degree in Computer Science from the University of Waterloo. His work at IBM focused on Computer Vision, Cloud Computing and Blockchain.</p>
<p><a href="https://www.linkedin.com/in/yi-leng-yao-84451275/" target="_blank">Yi Yao</a> is a Data Scientist and Software Engineer at IBM, and holds a Masters in Statistics. His research focused on Cloud Computing, Machine Learning and Computer Vision.</p>

<hr>
<p>Copyright &copy; 2019 IBM Developer Skills Network. This notebook and its source code are released under the terms of the <a href="https://cognitiveclass.ai/mit-license/">MIT License</a>.</p>